# Models

The most frequent usage for graph convolutions is either node or graph classification. As for their size, either a single large graph, e.g. citation network or small (batched) graphs like molecules have to be considered. 

Graphs can be represented by an index list of connections plus feature information. Typical quantities in tensor format to describe a graph are listed below.

* `nodes`: Node-list of shape `(batch, [N], F)` where `N` is the number of nodes and `F` is the node feature dimension.
* `edges`: Edge-list of shape `(batch, [M], F)` where `M` is the number of edges and `F` is the edge feature dimension.
* `indices`: Connection-list of shape `(batch, [M], 2)` where `M` is the number of edges. The indices denote a connection of incoming or receiving node `i` and outgoing or sending node `j` as `(i, j)`.
* `state`: Graph state information of shape `(batch, F)` where `F` denotes the feature dimension.
 
A major issue for graphs is their flexible size and shape, when using mini-batches. 
Here in `kgcnn` , for a graph implementation in the spirit of keras, the batch dimension should be kept also in between layers. This is realized by using `RaggedTensor` for the graph properties and indices.

Graph tensors for edge-indices or attributes for multiple graphs is passed to the model in form of ragged tensors 
of shape `(batch, None, Dim)` where `Dim` denotes a fixed feature or index dimension.
Such a ragged tensor has `ragged_rank=1` with one ragged dimension indicated by `None` and is build from a value plus partition tensor.

For example, the graph structure is represented by an index-list of shape `(batch, None, 2)` with index of incoming or receiving node `i` and outgoing or sending node `j` as `(i, j)`.
Note, an additional edge with `(j, i)` is required for undirected graphs. 
A ragged constant can be easily created and passed to a model either with `tf.RaggedTensor` methods or via a simple `tf.ragged.constant` :

In [1]:
import tensorflow as tf
import numpy as np
nodes = [[[0.0, 1.0, 0.0], [0.0, 0.0, 2.0]], [[0.0, 1.0, 2.0], [0.0, 0.0, 0.0], [1.0, 1.0, 3.0]], [[0.0, 0.0, 0.0]]]
idx = [[[0, 1], [1, 0]], [[0, 1], [1, 2], [2, 0]], [[0, 0]]]
labels = [[1.0], [0.0], [0.0]]  # batch_size=3

# Get ragged tensor of shape (3, None, 3) for node_input
node_input = tf.ragged.constant(nodes, ragged_rank=1, inner_shape=(3, ), dtype="float32")
node_input = tf.RaggedTensor.from_row_lengths(np.concatenate(nodes, dtype="float32"), [len(i) for i in nodes])
print(node_input.shape, node_input.dtype)

# Get ragged tensor of shape (3, None, 2) for indices
edge_index_input = tf.ragged.constant(idx, ragged_rank=1, inner_shape=(2, ), dtype="int64")
edge_index_input = tf.RaggedTensor.from_row_lengths(np.concatenate(idx, dtype="int64"), [len(i) for i in idx])
print(edge_index_input.shape, edge_index_input.dtype)

# Labels. No ragged dimension needed.
graph_labels = tf.constant(labels, dtype="float32")
print(graph_labels.shape, graph_labels.dtype)

(3, None, 3) <dtype: 'float32'>
(3, None, 2) <dtype: 'int64'>
(3, 1) <dtype: 'float32'>


## Functional API

Like most models in `kgcnn.literature` the models can be set up with the `tf.keras` functional API. Here an example for a simple message passing GNN. The layers are taken from `kgcnn.layers` . See documentation of layers for further details.

Layers

In [2]:
from kgcnn.layers.gather import GatherNodesSelection
from kgcnn.layers.modules import Dense, LazyConcatenate  # ragged support
from kgcnn.layers.pooling import PoolingLocalMessages, PoolingNodes

ks = tf.keras

n = ks.layers.Input(shape=(None, 3), name='node_input', dtype="float32", ragged=True)
ei = ks.layers.Input(shape=(None, 2), name='edge_index_input', dtype="int64", ragged=True)

n_in, n_out  = GatherNodesSelection([0, 1])([n, ei])
node_messages = Dense(10, activation='relu')(n_out)
node_updates = PoolingLocalMessages()([n, node_messages, ei])
n_node_updates = LazyConcatenate(axis=-1)([n, node_updates])
n_embedding = Dense(1)(n_node_updates)
g_embedding = PoolingNodes()(n_embedding)

message_functional = ks.models.Model(inputs=[n, ei], outputs=g_embedding)
message_functional.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 node_input (InputLayer)        [(None, None, 3)]    0           []                               
                                                                                                  
 edge_index_input (InputLayer)  [(None, None, 2)]    0           []                               
                                                                                                  
 gather_embedding_selection (Ga  [(None, None, 3),   0           ['node_input[0][0]',             
 therEmbeddingSelection)         (None, None, 3)]                 'edge_index_input[0][0]']       
                                                                                                  
 dense_embedding (DenseEmbeddin  (None, None, 10)    40          ['gather_embedding_selection[

In [3]:
out = message_functional.predict([node_input, edge_index_input])
print(out.shape)

1/1 [==============================] - 1s 654ms/step
(3, 1)


## Subclassing Model

A model can be constructed by subclassing from `tf.keras.Model` where the call method must be implemented. 

In [4]:
class SimpleGNN(tf.keras.Model):

    def __init__(self, units=10):
        super(SimpleGNN, self).__init__()
        self.gather = GatherNodesSelection([0, 1])
        self.message = Dense(units, activation='relu')
        self.aggregate = PoolingLocalMessages()
        self.concat = LazyConcatenate(axis=-1)
        self.classifier = Dense(1)
        self.pooling = PoolingNodes()

    def call(self, inputs):
        n, ei = inputs
        n_in, n_out = self.gather([n, ei])
        node_messages = self.message(n_out)
        node_updates = self.aggregate([n, node_messages, ei])
        n_node_updates = self.concat([n, node_updates])
        n_embedding = self.classifier(n_node_updates)
        g_embedding = self.pooling(n_embedding)
        return g_embedding

In [5]:
model_subclass = SimpleGNN()
out = model_subclass.predict([node_input, edge_index_input])
print(out.shape)

1/1 [==============================] - 0s 342ms/step
(3, 1)


Also layers can be further subclassed to create a GNN, for example of the message passing base layer. Where only `message_function` and `update_nodes` must be implemented.

In [6]:
from kgcnn.layers.message import MessagePassingBase
from kgcnn.layers.modules import Dense

class MyMessageNN(MessagePassingBase):

    def __init__(self, units=10, **kwargs):
        super(MyMessageNN, self).__init__(**kwargs)
        self.dense = Dense(units)
        self.add = LazyConcatenate()

    def message_function(self, inputs, **kwargs):
        n_in, n_out, edges = inputs
        return self.dense(n_out)

    def update_nodes(self, inputs, **kwargs):
        nodes, nodes_update = inputs
        return self.add([nodes, nodes_update])

In [7]:
# Note MyMessageNN is type only type layer.
message_layer = MyMessageNN()
out = message_layer([node_input, _, edge_index_input])
out.shape

TensorShape([3, None, 13])

## Loading options

There are many options to load data to a keras model, which depend on the size and location of the data to pass to the model. There may differences in speed and utility depending on the loading method. For more examples, please find https://github.com/aimat-lab/gcnn_keras/blob/master/notebooks/tutorial_model_loading_options.ipynb .

In [8]:
input_config = [
    {"shape": (None, 3), "name": "node_input", "dtype": "float32", "ragged": True},
    {"shape": (None, 2), "name": "edge_index_input", "dtype": "int64", "ragged": True}
]
output_config = {"shape": [], "name": "graph_labels", "dtype": "float32", "ragged": False}

In [9]:
model = message_functional
model.compile(loss="mean_absolute_error")

#### 1. Tensor Input

Tensor constants like the example above can be used as model input for data that comfortably fits into memory.
The `kgcnn.data.base.MemoryGraphList` has a `tensor()` method to generate a tensor from a list of properties.

In [10]:
from kgcnn.data.base import MemoryGraphList
dataset = MemoryGraphList([{"node_input": n, "edge_index_input": ei, "graph_labels": g} for n, ei, g in zip(nodes, idx, labels)])
print(dataset)

<MemoryGraphList [{'node_input': array([[0., 1., 0.],
       [0., 0., 2.]]), 'edge_index_input': array([[0, 1],
       [1, 0]]), 'graph_labels': array([1.])} ...]>


In [11]:
tensor_input = dataset.tensor(input_config)
tensor_output = dataset.tensor(output_config)
out = model.fit([node_input, edge_index_input], tensor_output)

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/pooling_local_edges/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/pooling_local_edges/Reshape:0", shape=(None, 10), dtype=float32), dense_shape=Tensor("gradient_tape/model/pooling_local_edges/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


1/1 [==============================] - 1s 653ms/step - loss: 0.7880


#### 2. Keras Sequence

For example `GraphBatchLoader` that inherits from `ks.utils.Sequence` and takes an iterable data object of type `list[dict]`.

In [12]:
from kgcnn.io.loader import GraphBatchLoader
loader = GraphBatchLoader(data=dataset, inputs=input_config, outputs=output_config)

In [13]:
out = model.fit(loader)

1/1 [==============================] - 0s 392ms/step - loss: 0.7661


####  3. TF Data

With tensorflow data and datasets. Again assuming given a list of indices and node properties.

* `from_tensor_slices`

In [14]:
ds_x = tf.data.Dataset.from_tensor_slices((
    tf.ragged.constant(nodes, ragged_rank=1, dtype="float32"),
    tf.ragged.constant(idx, ragged_rank=1, dtype="int64")))
ds_y = tf.data.Dataset.from_tensor_slices(tf.constant(labels))
ds = tf.data.Dataset.zip((ds_x, ds_y))
ds.batch(3)

<BatchDataset element_spec=((RaggedTensorSpec(TensorShape([None, None, 3]), tf.float32, 1, tf.int64), RaggedTensorSpec(TensorShape([None, None, 2]), tf.int64, 1, tf.int64)), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [15]:
model.fit(ds.batch(3), epochs=1)

1/1 [==============================] - 0s 8ms/step - loss: 0.7506


* `from_generator`

In [16]:
batch_size = 3
data_length = 3
def gen():
    for i in range(0, data_length, batch_size):
        yield (tf.ragged.constant(nodes[i:i+batch_size], dtype="float32", ragged_rank=1), 
               tf.ragged.constant(idx[i:i+batch_size], dtype="int64", ragged_rank=1))
    
ds_x_batch = tf.data.Dataset.from_generator(
    gen,
    output_signature=(
        tf.RaggedTensorSpec(shape=(None, None, 3), ragged_rank=1, dtype="float32"),
        tf.RaggedTensorSpec(shape=(None, None, 2), ragged_rank=1, dtype="int64")
    )
)
ds_y_batch = tf.data.Dataset.from_tensor_slices(tf.constant(labels)).batch(batch_size)
ds_batch = tf.data.Dataset.zip((ds_x_batch, ds_y_batch))
ds_batch

<ZipDataset element_spec=((RaggedTensorSpec(TensorShape([None, None, 3]), tf.float32, 1, tf.int64), RaggedTensorSpec(TensorShape([None, None, 2]), tf.int64, 1, tf.int64)), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [17]:
model.fit(ds_batch, epochs=1)

1/1 [==============================] - 1s 909ms/step - loss: 0.7375


* `tf.data.experimental.dense_to_ragged_batch`

In [18]:
bath_size = 3
dataset_list = []

ds_node = tf.data.Dataset.from_generator(
    lambda: [tf.constant(x) for x in nodes], 
    output_signature=tf.TensorSpec(shape=(None, 3), dtype="float32")
).apply(tf.data.experimental.dense_to_ragged_batch(batch_size=bath_size, drop_remainder=False))
ds_edge = tf.data.Dataset.from_generator(
    lambda: [tf.constant(x) for x in idx], 
    output_signature=tf.TensorSpec(shape=(None, 2), dtype="int64")
).apply(tf.data.experimental.dense_to_ragged_batch(batch_size=bath_size, drop_remainder=False))


ds_x_batch = tf.data.Dataset.zip((ds_node, ds_edge))
ds_y_batch = tf.data.Dataset.from_tensor_slices(tf.constant(graph_labels)).batch(batch_size)

ds_batch = tf.data.Dataset.zip((ds_x_batch, ds_y_batch))
ds_batch

<ZipDataset element_spec=((RaggedTensorSpec(TensorShape([None, None, 3]), tf.float32, 1, tf.int64), RaggedTensorSpec(TensorShape([None, None, 2]), tf.int64, 1, tf.int64)), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [19]:
model.fit(ds_batch, epochs=1)

1/1 [==============================] - 0s 45ms/step - loss: 0.7258


or via explicit generator

In [20]:
def gen():
    for i in range(len(nodes)):
        yield nodes[i], idx[i]

ds_x_batch = tf.data.Dataset.from_generator(
    gen, output_signature=(tf.TensorSpec(shape=(None, 3), dtype="float32"),tf.TensorSpec(shape=(None, 2), dtype="int64"))
).apply(tf.data.experimental.dense_to_ragged_batch(batch_size=bath_size, drop_remainder=False))

ds_y_batch = tf.data.Dataset.from_tensor_slices(tf.constant(graph_labels)).batch(batch_size)

ds_batch = tf.data.Dataset.zip((ds_x_batch, ds_y_batch))
ds_batch

<ZipDataset element_spec=((RaggedTensorSpec(TensorShape([None, None, 3]), tf.float32, 1, tf.int64), RaggedTensorSpec(TensorShape([None, None, 2]), tf.int64, 1, tf.int64)), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [21]:
model.fit(ds_batch, epochs=1)

1/1 [==============================] - 0s 39ms/step - loss: 0.7150


> **NOTE**: You can find this page as jupyter notebook in https://github.com/aimat-lab/gcnn_keras/tree/master/docs/source